# Time Mix Layer Reuse test

Starting from scratch

In [22]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="TLR-"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v6-TCLR/"))


print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

# Baseline layer count, and embedding size
L_SIZE=12
D_SIZE=512

# Deepspeed and batch size
DEEPSPEED_STAGE="deepspeed_stage_2"
BATCH_SIZE=4

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits
TRAINER_DIR: /home/recursal/RWKV-infctx-trainer/RWKV-v6-TCLR
PROJECT_DIR: /home/recursal/RWKV-infctx-trainer


In [8]:
# Init the model
!cd "{TRAINER_DIR}" && \
    python3 ./init_model.py \
        --n_layer {L_SIZE} --n_embd {D_SIZE} \
        --vocab_size world --skip-if-exists \
        "../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth"
!ls -lh "{TRAINER_DIR}/../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth"

[2024-04-21 06:18:50,078] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 1
[RWKV] CMIX reuse multiplier : 1
---- Initializing model ----
No of layers: 12
Embedding size: 512
Output model path: ../model/L12-D512-world-v6base-init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
Output model exists, skipping init_model
-rw-rw-r-- 1 recursal recursal 212M Apr 21 06:13 /home/recursal/RWKV-infctx-trainer/RWKV-v6-TCLR/../model/L12-D512-world-v6base-init.pth


# Dataset preloading

In [9]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml"

Saving the dataset (4/4 shards): 100%|█| 10020/10020 [00:06<00:00, 1491.65 examp
Saving the dataset (1/1 shards): 100%|████| 2/2 [00:00<00:00, 124.56 examples/s]


# Reference Run

In [25]:
# Configure the LRX experiment count
TLR_COUNT=1
CLR_COUNT=1

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_TMIX_REUSE_MULTIPLIER="{TLR_COUNT}" && \
    export RWKV_CMIX_REUSE_MULTIPLIER="{CLR_COUNT}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-TLR{TLR_COUNT}-CLR{CLR_COUNT}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}, TLR={TLR_COUNT}, CLR={CLR_COUNT} (Rechunk 32k {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found


[2024-04-21 08:51:21,934] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 1
[RWKV] CMIX reuse multiplier : 1
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L12-D512-world-v6base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-100k-L12-D512-TLR1-CLR1/', '--trainer.logger.init_args.name=TLR-L12-D512, TLR=1, CLR=1 (Rechunk 32k deepspeed_stage_2)', '--trainer.strategy=deepspeed_stage_2', '--trainer.m

# TLR=2

In [26]:
# Configure the LRX experiment count
TLR_COUNT=2
CLR_COUNT=1

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_TMIX_REUSE_MULTIPLIER="{TLR_COUNT}" && \
    export RWKV_CMIX_REUSE_MULTIPLIER="{CLR_COUNT}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-TLR{TLR_COUNT}-CLR{CLR_COUNT}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}, TLR={TLR_COUNT}, CLR={CLR_COUNT} (Rechunk 32k {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found


[2024-04-21 09:16:55,579] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 2
[RWKV] CMIX reuse multiplier : 1
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L12-D512-world-v6base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-100k-L12-D512-TLR2-CLR1/', '--trainer.logger.init_args.name=TLR-L12-D512, TLR=2, CLR=1 (Rechunk 32k deepspeed_stage_2)', '--trainer.strategy=deepspeed_stage_2', '--trainer.m

# TLR=4

In [27]:
# Configure the LRX experiment count
TLR_COUNT=4
CLR_COUNT=1

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_TMIX_REUSE_MULTIPLIER="{TLR_COUNT}" && \
    export RWKV_CMIX_REUSE_MULTIPLIER="{CLR_COUNT}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-TLR{TLR_COUNT}-CLR{CLR_COUNT}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}, TLR={TLR_COUNT}, CLR={CLR_COUNT} (Rechunk 32k {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found
[2024-04-21 09:41:50,221] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 4
[RWKV] CMIX reuse multiplier : 1
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L12-D512-world-v6base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-100k-L12-D512-TLR4-CLR1/', '--trainer.logger.init_args.name=TLR-L12-D512, TLR=4, CLR=1 (Rechunk 32k deepspeed_stage_2)', '--trainer.strategy=deepsp

# TLR=6

In [23]:
# Configure the LRX experiment count
TLR_COUNT=6
CLR_COUNT=1

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_TMIX_REUSE_MULTIPLIER="{TLR_COUNT}" && \
    export RWKV_CMIX_REUSE_MULTIPLIER="{CLR_COUNT}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-TLR{TLR_COUNT}-CLR{CLR_COUNT}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}, TLR={TLR_COUNT}, CLR={CLR_COUNT} (Rechunk 32k {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found


[2024-04-21 08:02:57,618] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 6
[RWKV] CMIX reuse multiplier : 1
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L12-D512-world-v6base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-100k-L12-D512-TLR6-CLR1/', '--trainer.logger.init_args.name=TLR-L12-D512, TLR=6, CLR=1 (Rechunk 32k deepspeed_stage_2)', '--trainer.strategy=deepspeed_stage_2', '--trainer.m

# TLR=12

In [24]:
# Configure the LRX experiment count
TLR_COUNT=12
CLR_COUNT=1

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_TMIX_REUSE_MULTIPLIER="{TLR_COUNT}" && \
    export RWKV_CMIX_REUSE_MULTIPLIER="{CLR_COUNT}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-TLR{TLR_COUNT}-CLR{CLR_COUNT}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}, TLR={TLR_COUNT}, CLR={CLR_COUNT} (Rechunk 32k {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found
[2024-04-21 08:27:02,552] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 12
[RWKV] CMIX reuse multiplier : 1
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L12-D512-world-v6base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-100k-L12-D512-TLR12-CLR1/', '--trainer.logger.init_args.name=TLR-L12-D512, TLR=12, CLR=1 (Rechunk 32k deepspeed_stage_2)', '--trainer.strategy=dee